In [1]:
import numpy as np 
import pandas as pd
import multiprocessing as mp
from multiprocessing import Pool
from numba import jit
import json

In [2]:
df = pd.read_csv('../datelets/datalet2.csv',index_col=False)

In [3]:
headers = []

for i in range(df.shape[1]):
    headers.append('period{}'.format(i))
 

df.columns = headers    

display(df)

,period0,period1
0,3.0,1.0
1,1.0,3.0
2,3.0,0.0
3,0.0,1.0
4,1.0,0.0
...,...,...
9993,3.0,1.0
9994,1.0,3.0
9995,3.0,3.0
9996,3.0,0.0


In [4]:
counts = {}
for x in range(len(df['period0'].value_counts())):
    counts['{}.0'.format(x)]= df['period0'].value_counts()[x]
    
print(counts)


{'0.0': 2518, '1.0': 2460, '2.0': 2552, '3.0': 2468}


In [5]:
df['ydata'] = df['period1']
output = pd.pivot_table(data=df, index=['period0'], columns=['period1'], values='ydata', aggfunc='count')
display(output)



period1,0.0,1.0,2.0,3.0
period0,,,,
0.0,593,633,671,621
1.0,656,578,611,615
2.0,641,649,630,632
3.0,629,600,640,599


In [6]:
#This is a the stochastic markov chain that tells you, if you start at a particular MEV state, where might you end up I am just transposing it for the calculation and it will be transposed back after

tmatrix = output.transpose(copy=False)

#This loop divides by the total counts in each row, to work out the conditional probability of going from one state to another. 

for x in range(len(counts)):
    tmatrix[x] = (tmatrix[x] / counts['{}.0'.format(x)])


transition_matrix = tmatrix.transpose(copy=False)

display(transition_matrix)

period1,0.0,1.0,2.0,3.0
period0,,,,
0.0,0.235504,0.251390,0.266481,0.246624
1.0,0.266667,0.234959,0.248374,0.250000
2.0,0.251176,0.254310,0.246865,0.247649
3.0,0.254862,0.243112,0.259319,0.242707
